# Regression Week 4: Ridge Regression (interpretation)

In this notebook, we will run ridge regression multiple times with different L2 penalties to see which one produces the best fit. We will revisit the example of polynomial regression as a means to see the effect of L2 regularization. In particular, we will:
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty using cross-validation.
* Assess the final fit using test data.

We will continue to use the House data from previous notebooks.  (In the next programming assignment for this module, you will implement your own ridge regression learning algorithm using gradient descent.)

# Fire up graphlab create

In [1]:
import graphlab

A newer version of GraphLab Create (v1.8) is available! Your current version is v1.7.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Polynomial regression, revisited

We build on the material from Week 3, where we wrote the function to produce an SFrame with columns containing the powers of a given input. Copy and paste the function `polynomial_sframe` from Week 3:

In [100]:
def polynomial_sframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe['power_1'] = feature
    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        for power in range(2, degree+1):
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # assign poly_sframe[name] to be feature^power
            poly_sframe[name] = feature.apply(lambda x: x**power)
    
    return poly_sframe

Let's use matplotlib to visualize what a polynomial regression looks like on the house data.

In [101]:
import matplotlib.pyplot as plt
%matplotlib inline

In [102]:
sales = graphlab.SFrame('kc_house_data.gl/')

In [104]:
poly1_data = polynomial_sframe(sales['sqft_living'], 15)

my_features = poly1_data.column_names()
poly1_data['price'] = sales['price']
my_features
poly1_data
polynomial_sframe(sales['sqft_living'], 15)

power_1,power_2,power_3,power_4,power_5,power_6,power_7
1180.0,1392400.0,1643032000.0,1.93877776e+12,2.2877577568e+15,2.69955415302e+18,3.18547390057e+21
2570.0,6604900.0,16974593000.0,4.362470401e+13,1.12115489306e+17,2.88136807516e+20,7.40511595315e+23
770.0,592900.0,456533000.0,3.5153041e+11,2.706784157e+14,2.08422380089e+17,1.60485232669e+20
1960.0,3841600.0,7529536000.0,1.475789056e+13,2.89254654976e+16,5.66939123753e+19,1.11120068256e+23
1680.0,2822400.0,4741632000.0,7.96594176e+12,1.33827821568e+16,2.24830740234e+19,3.77715643594e+22
5420.0,29376400.0,1.59220088e+11,8.6297287696e+14,4.67731299312e+18,2.53510364227e+22,1.37402617411e+26
1715.0,2941225.0,5044200875.0,8.65080450062e+12,1.48361297186e+16,2.54439624674e+19,4.36363956315e+22
1060.0,1123600.0,1191016000.0,1.26247696e+12,1.3382255776e+15,1.41851911226e+18,1.50363025899e+21
1780.0,3168400.0,5639752000.0,1.003875856e+13,1.78689902368e+16,3.18068026215e+19,5.66161086663e+22
1890.0,3572100.0,6751269000.0,1.275989841e+13,2.41162079949e+16,4.55796331104e+19,8.61455065786e+22


As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living. For houses with identical square footage, we break the tie by their prices.

In [105]:
sales = sales.sort(['sqft_living','price'])

Let us revisit the 15th-order polynomial model using the 'sqft_living' input. Generate polynomial features up to degree 15 using `polynomial_sframe()` and fit a model with these features. When fitting the model, use an L2 penalty of `1e-5`:

In [106]:
l2_small_penalty = 1e-5

Note: When we have so many features and so few data points, the solution can become highly numerically unstable, which can sometimes lead to strange unpredictable results.  Thus, rather than using no regularization, we will introduce a tiny amount of regularization (`l2_penalty=1e-5`) to make the solution numerically stable.  (In lecture, we discussed the fact that regularization can also help with numerical stability, and here we are seeing a practical example.)

With the L2 penalty specified above, fit the model and print out the learned weights.

Hint: make sure to add 'price' column to the new SFrame before calling `graphlab.linear_regression.create()`. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set=None` in this call.

In [107]:
model1 = graphlab.linear_regression.create(poly1_data, target = 'price', features = my_features,l2_penalty=l2_small_penalty, validation_set = None)
model1.coefficients

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Number of coefficients    : 16
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.056868     | 2662555.733758     | 245656.462162 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:


name,index,value
(intercept),None,167924.887498
power_1,None,103.090854766
power_2,None,0.134604670814
power_3,None,-0.000129071438543
power_4,None,5.18929212461e-08
power_5,None,-7.77169792098e-12
power_6,None,1.71145348923e-16
power_7,None,4.51177695997e-20
power_8,None,-4.78838676735e-25
power_9,None,-2.33343548826e-28


***QUIZ QUESTION:  What's the learned value for the coefficient of feature `power_1`?***

# Observe overfitting

Recall from Week 3 that the polynomial fit of degree 15 changed wildly whenever the data changed. In particular, when we split the sales data into four subsets and fit the model of degree 15, the result came out to be very different for each subset. The model had a *high variance*. We will see in a moment that ridge regression reduces such variance. But first, we must reproduce the experiment we did in Week 3.

First, split the data into split the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use `.random_split` function and make sure you set `seed=0`. 

In [115]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=0)
(set_1, set_2) = semi_split1.random_split(0.5, seed=0)
(set_3, set_4) = semi_split2.random_split(0.5, seed=0)

Next, fit a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Print the weights and make a plot of the resulting model.

Hint: When calling `graphlab.linear_regression.create()`, use the same L2 penalty as before (i.e. `l2_small_penalty`).  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [123]:
poly_data_2 = polynomial_sframe(set_1['sqft_living'], 15)
poly_data_2['price'] = set_1['price']
model2 = graphlab.linear_regression.create(poly_data_2, target = 'price',
                                          l2_penalty=l2_small_penalty, features = my_features, validation_set = None,verbose=False)
model2.coefficients

name,index,value
(intercept),None,9306.4606221
power_1,None,585.865823394
power_2,None,-0.397305895643
power_3,None,0.000141470900599
power_4,None,-1.52945989958e-08
power_5,None,-3.79756325772e-13
power_6,None,5.97481763253e-17
power_7,None,1.06888504767e-20
power_8,None,1.59344027887e-25
power_9,None,-6.92834984105e-29


In [122]:
poly_data_3 = polynomial_sframe(set_2['sqft_living'], 15)
poly_data_3['price'] = set_2['price']
model3 = graphlab.linear_regression.create(poly_data_3, target = 'price',
                                          l2_penalty=l2_small_penalty, features = my_features, validation_set = None,verbose=False)
model3.coefficients

name,index,value
(intercept),None,-25115.9044254
power_1,None,783.49380028
power_2,None,-0.767759302942
power_3,None,0.000438766369254
power_4,None,-1.15169166858e-07
power_5,None,6.84281360981e-12
power_6,None,2.51195187082e-15
power_7,None,-2.06440608259e-19
power_8,None,-4.59673022352e-23
power_9,None,-2.71279236963e-29


In [121]:
poly_data_4 = polynomial_sframe(set_3['sqft_living'], 15)

poly_data_4['price'] = set_3['price']
model4 = graphlab.linear_regression.create(poly_data_4, target = 'price',
                                          l2_penalty=l2_small_penalty, features = my_features, validation_set = None,verbose=False)
model4.coefficients

name,index,value
(intercept),None,462426.565731
power_1,None,-759.251842854
power_2,None,1.0286700473
power_3,None,-0.000528264527386
power_4,None,1.15422908385e-07
power_5,None,-2.26095948062e-12
power_6,None,-2.08214287571e-15
power_7,None,4.08770475709e-20
power_8,None,2.570791329e-23
power_9,None,1.24311265196e-27


In [120]:
poly_data_5 = polynomial_sframe(set_4['sqft_living'], 15)
poly_data_5['price'] = set_4['price']
model5 = graphlab.linear_regression.create(poly_data_5, target = 'price',
                                          l2_penalty=l2_small_penalty, features = my_features, validation_set = None,verbose=False)
model5.coefficients

name,index,value
(intercept),None,-170240.032842
power_1,None,1247.59034541
power_2,None,-1.22460912177
power_3,None,0.000555254626344
power_4,None,-6.3826237386e-08
power_5,None,-2.20215991142e-11
power_6,None,4.81834694285e-15
power_7,None,4.21461612787e-19
power_8,None,-7.99880736276e-23
power_9,None,-1.32365897487e-26


The four curves should differ from one another a lot, as should the coefficients you learned.

***QUIZ QUESTION:  For the models learned in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?***  (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Ridge regression comes to rescue

Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because 'sqft_living' input is in the order of thousands.)

With the argument `l2_penalty=1e5`, fit a 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`. Other than the change in the `l2_penalty` parameter, the code should be the same as the experiment above. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [129]:
l2_penalty=1e5

In [130]:
poly_data_2 = polynomial_sframe(set_1['sqft_living'], 15)
poly_data_2['price'] = set_1['price']
model2 = graphlab.linear_regression.create(poly_data_2, target = 'price',
                                          l2_penalty=l2_penalty, features = my_features, validation_set = None,verbose=False)
model2.coefficients

name,index,value
(intercept),None,530317.024516
power_1,None,2.58738875673
power_2,None,0.00127414400592
power_3,None,1.74934226932e-07
power_4,None,1.06022119097e-11
power_5,None,5.42247604482e-16
power_6,None,2.89563828343e-20
power_7,None,1.65000666351e-24
power_8,None,9.86081528409e-29
power_9,None,6.06589348254e-33


In [131]:
poly_data_3 = polynomial_sframe(set_2['sqft_living'], 15)
poly_data_3['price'] = set_2['price']
model3 = graphlab.linear_regression.create(poly_data_3, target = 'price',
                                          l2_penalty=l2_penalty, features = my_features, validation_set = None,verbose=False)
model3.coefficients

name,index,value
(intercept),None,519216.897383
power_1,None,2.04470474182
power_2,None,0.0011314362684
power_3,None,2.93074277549e-07
power_4,None,4.43540598453e-11
power_5,None,4.80849112204e-15
power_6,None,4.53091707826e-19
power_7,None,4.16042910575e-23
power_8,None,3.90094635128e-27
power_9,None,3.7773187602e-31


In [132]:
poly_data_4 = polynomial_sframe(set_3['sqft_living'], 15)

poly_data_4['price'] = set_3['price']
model4 = graphlab.linear_regression.create(poly_data_4, target = 'price',
                                          l2_penalty=l2_penalty, features = my_features, validation_set = None,verbose=False)
model4.coefficients

name,index,value
(intercept),None,522911.518048
power_1,None,2.26890421877
power_2,None,0.00125905041842
power_3,None,2.77552918155e-07
power_4,None,3.2093309779e-11
power_5,None,2.87573572364e-15
power_6,None,2.50076112671e-19
power_7,None,2.24685265906e-23
power_8,None,2.09349983135e-27
power_9,None,2.00435383296e-31


In [133]:
poly_data_5 = polynomial_sframe(set_4['sqft_living'], 15)
poly_data_5['price'] = set_4['price']
model5 = graphlab.linear_regression.create(poly_data_5, target = 'price',
                                          l2_penalty=l2_penalty, features = my_features, validation_set = None,verbose=False)
model5.coefficients

name,index,value
(intercept),None,513667.087087
power_1,None,1.91040938244
power_2,None,0.00110058029175
power_3,None,3.12753987879e-07
power_4,None,5.50067886825e-11
power_5,None,7.20467557825e-15
power_6,None,8.24977249384e-19
power_7,None,9.06503223498e-23
power_8,None,9.95683160453e-27
power_9,None,1.10838127982e-30


These curves should vary a lot less, now that you applied a high degree of regularization.

***QUIZ QUESTION:  For the models learned with the high level of regularization in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?*** (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Selecting an L2 penalty via cross-validation

Just like the polynomial degree, the L2 penalty is a "magic" parameter we need to select. We could use the validation set approach as we did in the last module, but that approach has a major disadvantage: it leaves fewer observations available for training. **Cross-validation** seeks to overcome this issue by using all of the training set in a smart way.

We will implement a kind of cross-validation called **k-fold cross-validation**. The method gets its name because it involves dividing the training set into k segments of roughtly equal size. Similar to the validation set method, we measure the validation error with one of the segments designated as the validation set. The major difference is that we repeat the process k times as follows:

Set aside segment 0 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
Set aside segment 1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
...<br>
Set aside segment k-1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set

After this process, we compute the average of the k validation errors, and use it as an estimate of the generalization error. Notice that  all observations are used for both training and validation, as we iterate over segments of data. 

To estimate the generalization error well, it is crucial to shuffle the training data before dividing them into segments. GraphLab Create has a utility function for shuffling a given SFrame. We reserve 10% of the data as the test set and shuffle the remainder. (Make sure to use `seed=1` to get consistent answer.)

In [135]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)

Once the data is shuffled, we divide it into equal segments. Each segment should receive `n/k` elements, where `n` is the number of observations in the training set and `k` is the number of segments. Since the segment 0 starts at index 0 and contains `n/k` elements, it ends at index `(n/k)-1`. The segment 1 starts where the segment 0 left off, at index `(n/k)`. With `n/k` elements, the segment 1 ends at index `(n*2/k)-1`. Continuing in this fashion, we deduce that the segment `i` starts at index `(n*i/k)` and ends at `(n*(i+1)/k)-1`.

With this pattern in mind, we write a short loop that prints the starting and ending indices of each segment, just to make sure you are getting the splits right.

In [136]:
n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

for i in xrange(k):
    start = (n*i)/k
    end = (n*(i+1))/k-1
    print i, (start, end)

0 (0, 1938)
1 (1939, 3878)
2 (3879, 5817)
3 (5818, 7757)
4 (7758, 9697)
5 (9698, 11636)
6 (11637, 13576)
7 (13577, 15515)
8 (15516, 17455)
9 (17456, 19395)


Let us familiarize ourselves with array slicing with SFrame. To extract a continuous slice from an SFrame, use colon in square brackets. For instance, the following cell extracts rows 0 to 9 of `train_valid_shuffled`. Notice that the first index (0) is included in the slice but the last index (10) is omitted.

In [137]:
train_valid_shuffled[0:10] # rows 0 to 9

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
2780400035,2014-05-05 00:00:00+00:00,665000.0,4.0,2.5,2800.0,5900,1,0
1703050500,2015-03-21 00:00:00+00:00,645000.0,3.0,2.5,2490.0,5978,2,0
5700002325,2014-06-05 00:00:00+00:00,640000.0,3.0,1.75,2340.0,4206,1,0
0475000510,2014-11-18 00:00:00+00:00,594000.0,3.0,1.0,1320.0,5000,1,0
0844001052,2015-01-28 00:00:00+00:00,365000.0,4.0,2.5,1904.0,8200,2,0
2658000373,2015-01-22 00:00:00+00:00,305000.0,4.0,2.0,1610.0,6250,1,0
3750603471,2015-03-27 00:00:00+00:00,239950.0,3.0,2.5,1560.0,4800,2,0
2114700540,2014-10-21 00:00:00+00:00,366000.0,3.0,2.5,1320.0,4320,1,0
2596400050,2014-07-30 00:00:00+00:00,375000.0,3.0,1.0,1960.0,7955,1,0
4140900050,2015-01-26 00:00:00+00:00,440000.0,4.0,1.75,2180.0,10200,1,0


Now let us extract individual segments with array slicing. Consider the scenario where we group the houses in the `train_valid_shuffled` dataframe into k=10 segments of roughly equal size, with starting and ending indices computed as above.
Extract the fourth segment (segment 3) and assign it to a variable called `validation4`.

In [ ]:
0 (0, 1938)
1 (1939, 3878)
2 (3879, 5817)
3 (5818, 7757)
4 (7758, 9697)
5 (9698, 11636)
6 (11637, 13576)
7 (13577, 15515)
8 (15516, 17455)
9 (17456, 19395)

In [254]:
validation4 = train_valid_shuffled[5818:7758]
validation4[['price']][1939]


{'price': 306500.0}

To verify that we have the right elements extracted, run the following cell, which computes the average price of the fourth segment. When rounded to nearest whole number, the average should be $536,234.

In [291]:
print int(round(validation4['price'].mean(), 0))

536234


After designating one of the k segments as the validation set, we train a model using the rest of the data. To choose the remainder, we slice (0:start) and (end+1:n) of the data and paste them together. SFrame has `append()` method that pastes together two disjoint sets of rows originating from a common dataset. For instance, the following cell pastes together the first and last two rows of the `train_valid_shuffled` dataframe.

In [307]:
n = len(train_valid_shuffled)
first = train_valid_shuffled[0:5819]
last = train_valid_shuffled[n-7758:n]
train4 = first.append(last)

Extract the remainder of the data after *excluding* fourth segment (segment 3) and assign the subset to `train4`.

In [308]:
len(train4)

13577

To verify that we have the right elements extracted, run the following cell, which computes the average price of the data with fourth segment excluded. When rounded to nearest whole number, the average should be $539,450.

In [309]:
print int(round(train4['price'].mean(), 0))

537484


Now we are ready to implement k-fold cross-validation. Write a function that computes k validation errors by designating each of the k segments as the validation set. It accepts as parameters (i) `k`, (ii) `l2_penalty`, (iii) dataframe, (iv) name of output column (e.g. `price`) and (v) list of feature names. The function returns the average validation error using k segments as validation sets.

* For each i in [0, 1, ..., k-1]:
  * Compute starting and ending indices of segment i and call 'start' and 'end'
  * Form validation set by taking a slice (start:end+1) from the data.
  * Form training set by appending slice (end+1:n) to the end of slice (0:start).
  * Train a linear model using training set just formed, with a given l2_penalty
  * Compute validation error using validation set just formed

In [ ]:
def k_fold_cross_validation(k, l2_penalty, data, output_name, features_list):
    

Once we have a function to compute the average validation error for a model, we can write a loop to find the model that minimizes the average validation error. Write a loop that does the following:
* We will again be aiming to fit a 15th-order polynomial model using the `sqft_living` input
* For `l2_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, you can use this Numpy function: `np.logspace(1, 7, num=13)`.)
    * Run 10-fold cross-validation with `l2_penalty`
* Report which L2 penalty produced the lowest average validation error.

Note: since the degree of the polynomial is now fixed to 15, to make things faster, you should generate polynomial features in advance and re-use them throughout the loop. Make sure to use `train_valid_shuffled` when generating polynomial features!

***QUIZ QUESTIONS:  What is the best value for the L2 penalty according to 10-fold validation?***

You may find it useful to plot the k-fold cross-validation errors you have obtained to better understand the behavior of the method.  

In [ ]:
# Plot the l2_penalty values in the x axis and the cross-validation error in the y axis.
# Using plt.xscale('log') will make your plot more intuitive.



Once you found the best value for the L2 penalty using cross-validation, it is important to retrain a final model on all of the training data using this value of `l2_penalty`.  This way, your final model will be trained on the entire dataset.

***QUIZ QUESTION: Using the best L2 penalty found above, train a model using all training data. What is the RSS on the TEST data of the model you learn with this L2 penalty? ***